In [ ]:
import hashlib
import os
import pickle
from hwcomponents_cacti import SRAM
from hwcomponents_library import AladdinAdder, AladdinMultiplier

from fastfusion.frontend.architecture import Memory
from fastfusion.frontend.specification import Specification
from fastfusion.mapper.FFM.exploration.mapper_multi_einsum import get_sims
from fastfusion.mapper.simanneal.wrappers import join_sims

import copy
import time
from fastfusion import Specification
from fastfusion.mapper.FFM.exploration.metrics import Metrics
from fastfusion.mapper.FFM.exploration.mapper_multi_einsum import get_sims
from fastfusion.mapper.FFM.joining.sim import SIM
from fastfusion.mapper.FFM.joining.simexplore import join_sims
import fastfusion.mapper.FFM.exploration.mapper_one_einsum as mapper_one_einsum

from fastfusion.mapper.FFM.exploration.mapping_filter_tags.ffmt import get_ffmt_tag
from fastfusion.mapper.FFM.exploration.mapping_filter_tags.onesplit import get_one_split_tag
from fastfusion.mapper.FFM.pareto import PartialMappings

spec = Specification.from_yaml(
    f"architecture/four_level.arch.yaml",
    "workloads/mha_full.workload.yaml",
    "workloads/mha_full.renames.yaml",
)

# adder = AladdinAdder(technology="7nm", width=16)
# multiplier = AladdinMultiplier(technology="7nm", width=8)
# mac_area = adder.get_area() + multiplier.get_area()

# base_local_buffer_size = 4 * 1024 * 1024 * 8
# base_local_buffer = CactiSRAM(technology="7nm", width=128, depth=base_local_buffer_size // 128)
# base_global_buffer_size = 128 * 1024 * 1024 * 8
# base_global_buffer = CactiSRAM(technology="7nm", width=1024, depth=base_global_buffer_size // 1024)
# area_budget = (mac_area * 128 * 128 + base_local_buffer.get_area()) * 4 + base_global_buffer.get_area()

# TARGET_TECH_NODE = "4nm"
# adder = AladdinAdder(technology=TARGET_TECH_NODE, width=16)
# multiplier = AladdinMultiplier(technology=TARGET_TECH_NODE, width=8)
# mac_area = adder.get_area() + multiplier.get_area()
# base_local_buffer_size = 1 * 1024 * 1024 * 8
# base_local_buffer = CactiSRAM(technology=TARGET_TECH_NODE, width=128, depth=base_local_buffer_size // 128)
# base_global_buffer_size = 16.5 * 1024 * 1024 * 8
# base_global_buffer = CactiSRAM(technology=TARGET_TECH_NODE, width=1024, depth=base_global_buffer_size // 1024)
# area_budget = (mac_area * 512 * 1024 + base_local_buffer.get_area()) * 4 + base_global_buffer.get_area()

print(f"COMPUTE ENERGY / 8 ????????????")
print(f"COMPUTE ENERGY / 8 ????????????")
print(f"COMPUTE ENERGY / 8 ????????????")
print(f"COMPUTE ENERGY / 8 ????????????")
print(f"COMPUTE ENERGY / 8 ????????????")

objective = lambda df: df['metric_Latency']# * df['metric_Energy']

def get_fused_mappings(
        spec: Specification,
        n_pes,
        pe_x,
        pe_y,
        tagger=None, 
        # fuse=True,
        parameterization="",
        return_mappings=False,
        max_latency: float = None
    ) -> PartialMappings:
    print(f'Running parameterization {parameterization}')
    cachekey = (n_pes, pe_x, pe_y, parameterization)
    fname = parameterization + " " + hashlib.md5(str(cachekey).encode()).hexdigest()
    if os.path.exists(f"cache/{fname}.pkl") and parameterization != "FlashAttention B" and parameterization != "FFM":
        print(f"Loading from cache: {fname}")
        mappings = pickle.load(open(f"cache/{fname}.pkl", "rb"))
        if return_mappings:
            return mappings
        return objective(mappings.data).min(), mappings
    spec = copy.deepcopy(spec)
    # local_buffer: Memory = spec.architecture.nodes["LocalBuffer"]
    # local_buffer.attributes.size = local_buffer_model.width * local_buffer_model.depth
    # local_buffer.spatial.fanout["X"] = n_pes
    global_buffer: Memory = spec.architecture.nodes["GlobalBuffer"]
    main_memory: Memory = spec.architecture.nodes["MainMemory"]
    if parameterization == "Unfused":
        main_memory.constraints.storage.keep = "All()"
    elif parameterization == "FlashAttention B":
        main_memory.constraints.storage.keep = "All() - (Q | K | QK | QK_softmax)"# - QK_softmax"# - Q - K - V - I"
        main_memory.constraints.storage.bypass = "Q | K | QK | QK_softmax"#Q | K | V | I"# | QK | FFA"
    elif parameterization == "FlashAttention A":
        main_memory.constraints.storage.keep = "All() - (QK | QK_softmax)"# - QK_softmax"# - Q - K - V - I"
        main_memory.constraints.storage.bypass = "QK | QK_softmax"#Q | K | V | I"# | QK | FFA"
    elif parameterization == "FFM":
        main_memory.constraints.storage.keep = "~Intermediates()" #"# | AV | Z "
        # main_memory.constraints.storage.bypass = "I | Q | K | V | QK"#Q | K | V | I"# | QK | FFA"
        pass
    elif parameterization == "Weight-Stationary":
        main_memory.constraints.storage.keep = "~Intermediates()" #"# | AV | Z "
        global_buffer.constraints.dataflow.storage_orders.append(["weight", "input", "output"])
    elif parameterization == "Input-Stationary":
        global_buffer.constraints.dataflow.storage_orders.append(["input", "output", "weight"])
    elif parameterization == "Output-Stationary":
        global_buffer.constraints.dataflow.storage_orders.append(["output", "weight", "input"])
    elif parameterization == "Fixed-Dataflow":
        main_memory.constraints.storage.keep = "~Intermediates() | weight"
        global_buffer.constraints.dataflow.storage_orders.append([
            "MainMemory.tensors() & weight", 
            "MainMemory.tensors() & input", 
            "MainMemory.tensors() & output", 
            "~MainMemory.tensors() & weight", 
            "~MainMemory.tensors() & input", 
            "~MainMemory.tensors() & output",
        ])
    else:
        assert False, f"Parameterization {parameterization} not supported"
        
    register: Memory = spec.architecture.nodes["Register"]
    register.spatial.fanout["X"] = pe_x
    register.spatial.fanout["Y"] = pe_y
    register.spatial.fanout["Z"] = n_pes
    
    spec.estimate_energy_area()
    flattened_architecture = spec.get_flattened_architecture()
    t0 = time.time()
    sims, decompress_data = get_sims(spec, flattened_architecture, tagger=tagger, metrics=Metrics.LATENCY | Metrics.ENERGY) # metrics=Metrics.ENERGY | # | Metrics.PER_COMPONENT_ENERGY)
    pmapping_time = time.time() - t0
    total_pmappings = sum(p.mappings.n_pmappings for v in sims.values() for p in v)
    n_pareto_optimal_mappings = sum(len(p.mappings.data) for v in sims.values() for p in v)
    print(f'Took {pmapping_time:.2f} seconds to generate {total_pmappings} partial mappings ({total_pmappings / pmapping_time:.2f} per second). {n_pareto_optimal_mappings} pareto optimal mappings ({n_pareto_optimal_mappings / total_pmappings*100:.2f}% of total).')
    t0 = time.time()
    mappings = join_sims(sims, spec, flattened_architecture)
    join_time = time.time() - t0
    mappings.decompress(decompress_data)
    print(f"Pmappings: {pmapping_time:.2f}. Joining: {join_time:.2f}. Total Pmappings: {total_pmappings}. Total mappings: {mappings.n_pmappings}. Time per pmapping: {pmapping_time / total_pmappings:.2e}")
    pickle.dump(mappings, open(f"cache/{fname}.pkl", "wb"))
    if return_mappings:
        return mappings
    return objective(mappings.data).min(), mappings

# print(f'Overall area budget: {area_budget * 1e6} mm^2')

parameterization2edp = {}
parameterization2mappings = {}

parameterizations = [
    "Unfused",
    "Fixed-Dataflow",
    "FlashAttention A",
    "FlashAttention B",
    "FFM"
]#, "Unfused"] # "FFM", "Unfused", "FlashAttention"]#, "FlashAttention", "Unfused"]

# TARGET_TECH_NODE = "4nm"
# adder = AladdinAdder(technology=TARGET_TECH_NODE, width=16)
# multiplier = AladdinMultiplier(technology=TARGET_TECH_NODE, width=8)
# mac_area = adder.get_area() + multiplier.get_area()

# # glb_size = 512 * 1024 * 1024 * 8
# # glb = CactiSRAM(technology=TARGET_TECH_NODE, width=1024, depth=glb_size // 1024)
# llb_size = 1 * 1024 * 1024 * 8
# llb = CactiSRAM(technology=TARGET_TECH_NODE, width=128, depth=llb_size // 128)

def binary_search_glb_size(area_remaining):
    print(f"Area remaining: {area_remaining}")
    # Binary search to nearest 1MB
    low = 1
    high = 1024
    while low < high:
        mid = round((low + high) / 2)
        size = mid * 1024 * 1024 * 8
        glb = CactiSRAM(technology=TARGET_TECH_NODE, width=1024, depth=size // 1024)
        if glb.get_area() > area_remaining:
            if mid == high:
                break
            print(f"High {high} -> {mid}")
            high = mid
        else:
            if mid == low:
                break
            print(f"Low {low} -> {mid}")
            low = mid
    return low * 1024 * 1024 * 8

mac_x, mac_y = 128, 128
# for mac_x, mac_y in [(256, 256), (1024, 1024), (1024, 512), (512, 512), (512, 256)]:
# for n_pes in [256, 1, 2, 4, 8, 16, 32, 64, 128, 256]:#2, 4, 8, 16, 32]:
for n_pes in [256, 64]:#
    # total_mac_area = mac_area * mac_x * mac_y
    # area_remaining = area_budget - n_pes * (llb.get_area() + total_mac_area)
    # if area_remaining < 0:
    #     continue
    # glb_size = binary_search_glb_size(area_remaining)
    # glb = CactiSRAM(technology=TARGET_TECH_NODE, width=1024, depth=glb_size // 1024)
    # glb_size = 128 * 1024 * 1024 * 8
    # glb = CactiSRAM(technology=TARGET_TECH_NODE, width=1024, depth=glb_size // 1024)
    max_latency = None
    
    # glb_MB = glb_size // 1024 // 1024 // 8
    # llb_MB = llb_size // 1024 // 1024 // 8
    
    print(f"\n\n")
    print(f"=" * 100)
    print(f"MAC dims: {mac_x}x{mac_y}")
    print(f"=" * 100)

    for parameterization in parameterizations: # "fuse"
        while True:
            try:
                x, mappings = get_fused_mappings(
                    spec,
                    n_pes,
                    mac_x,
                    mac_y,
                    parameterization=parameterization,
                    max_latency=max_latency if parameterization != "FlashAttention B" else None
                )
                break
            except Exception as e:
                max_latency *= 2
                print(f"Error: {e}")
        if "Fixed-Dataflow" not in parameterization:
            max_latency = x
        if x != 0:
            parameterization2edp[f"{parameterization} {n_pes} {mac_x}x{mac_y}"] = x
            parameterization2mappings[f"{parameterization} {n_pes} {mac_x}x{mac_y}"] = mappings
            
            
            
            

spec = Specification.from_yaml(
    f"architecture/four_level_no_weight_par.arch.yaml",
    "workloads/mha_full_long_sequence.workload.yaml",
    "workloads/mha_full.renames.yaml",
)

# adder = AladdinAdder(technology="7nm", width=16)
# multiplier = AladdinMultiplier(technology="7nm", width=8)
# mac_area = adder.get_area() + multiplier.get_area()

# base_local_buffer_size = 4 * 1024 * 1024 * 8
# base_local_buffer = CactiSRAM(technology="7nm", width=128, depth=base_local_buffer_size // 128)
# base_global_buffer_size = 128 * 1024 * 1024 * 8
# base_global_buffer = CactiSRAM(technology="7nm", width=1024, depth=base_global_buffer_size // 1024)
# area_budget = (mac_area * 128 * 128 + base_local_buffer.get_area()) * 4 + base_global_buffer.get_area()

# TARGET_TECH_NODE = "4nm"
# adder = AladdinAdder(technology=TARGET_TECH_NODE, width=16)
# multiplier = AladdinMultiplier(technology=TARGET_TECH_NODE, width=8)
# mac_area = adder.get_area() + multiplier.get_area()
# base_local_buffer_size = 1 * 1024 * 1024 * 8
# base_local_buffer = CactiSRAM(technology=TARGET_TECH_NODE, width=128, depth=base_local_buffer_size // 128)
# base_global_buffer_size = 16.5 * 1024 * 1024 * 8
# base_global_buffer = CactiSRAM(technology=TARGET_TECH_NODE, width=1024, depth=base_global_buffer_size // 1024)
# area_budget = (mac_area * 512 * 1024 + base_local_buffer.get_area()) * 4 + base_global_buffer.get_area()

print(f"COMPUTE ENERGY / 8 ????????????")
print(f"COMPUTE ENERGY / 8 ????????????")
print(f"COMPUTE ENERGY / 8 ????????????")
print(f"COMPUTE ENERGY / 8 ????????????")
print(f"COMPUTE ENERGY / 8 ????????????")

objective = lambda df: df['metric_Latency']# * df['metric_Energy']

def get_fused_mappings(
        spec: Specification,
        n_pes,
        pe_x,
        pe_y,
        tagger=None, 
        # fuse=True,
        parameterization="",
        return_mappings=False,
        max_latency: float = None
    ) -> PartialMappings:
    print(f'Running parameterization {parameterization}')
    cachekey = (n_pes, pe_x, pe_y, parameterization)
    fname = parameterization + " " + hashlib.md5(str(cachekey).encode()).hexdigest()
    if os.path.exists(f"cache2/{fname}.pkl") and parameterization != "FlashAttention B":
        print(f"Loading from cache2: {fname}")
        mappings = pickle.load(open(f"cache2/{fname}.pkl", "rb"))
        if return_mappings:
            return mappings
        return objective(mappings.data).min(), mappings
    spec = copy.deepcopy(spec)
    # local_buffer: Memory = spec.architecture.nodes["LocalBuffer"]
    # local_buffer.attributes.size = local_buffer_model.width * local_buffer_model.depth
    # local_buffer.spatial.fanout["X"] = n_pes
    global_buffer: Memory = spec.architecture.nodes["GlobalBuffer"]
    main_memory: Memory = spec.architecture.nodes["MainMemory"]
    if parameterization == "Unfused":
        main_memory.constraints.storage.keep = "All()"
    elif parameterization == "FlashAttention B":
        main_memory.constraints.storage.keep = "All() - (Q | K | QK | QK_softmax)"# - QK_softmax"# - Q - K - V - I"
        main_memory.constraints.storage.bypass = "Q | K | QK | QK_softmax"#Q | K | V | I"# | QK | FFA"
    elif parameterization == "FlashAttention A":
        main_memory.constraints.storage.keep = "All() - (QK | QK_softmax)"# - QK_softmax"# - Q - K - V - I"
        main_memory.constraints.storage.bypass = "QK | QK_softmax"#Q | K | V | I"# | QK | FFA"
    elif parameterization == "FFM":
        main_memory.constraints.storage.keep = "~Intermediates() - (I | Q | K | V | QK | QK_softmax)" #"# | AV | Z "
        # main_memory.constraints.storage.bypass = "I | Q | K | V | QK | QK_softmax"#Q | K | V | I"# | QK | FFA"
        pass
    elif parameterization == "Weight-Stationary":
        main_memory.constraints.storage.keep = "~Intermediates()" #"# | AV | Z "
        global_buffer.constraints.dataflow.storage_orders.append(["weight", "input", "output"])
    elif parameterization == "Input-Stationary":
        global_buffer.constraints.dataflow.storage_orders.append(["input", "output", "weight"])
    elif parameterization == "Output-Stationary":
        global_buffer.constraints.dataflow.storage_orders.append(["output", "weight", "input"])
    elif parameterization == "Fixed-Dataflow":
        main_memory.constraints.storage.keep = "~Intermediates() | weight"
        global_buffer.constraints.dataflow.storage_orders.append([
            "MainMemory.tensors() & weight", 
            "MainMemory.tensors() & input", 
            "MainMemory.tensors() & output", 
            "~MainMemory.tensors() & weight", 
            "~MainMemory.tensors() & input", 
            "~MainMemory.tensors() & output",
        ])
    else:
        assert False, f"Parameterization {parameterization} not supported"
        
    register: Memory = spec.architecture.nodes["Register"]
    register.spatial.fanout["X"] = pe_x
    register.spatial.fanout["Y"] = pe_y
    register.spatial.fanout["Z"] = n_pes
    
    spec.estimate_energy_area()
    flattened_architecture = spec.get_flattened_architecture()
    t0 = time.time()
    sims, decompress_data = get_sims(spec, flattened_architecture, tagger=tagger, metrics=Metrics.LATENCY | Metrics.ENERGY) # metrics=Metrics.ENERGY | # | Metrics.PER_COMPONENT_ENERGY)
    pmapping_time = time.time() - t0
    total_pmappings = sum(p.mappings.n_pmappings for v in sims.values() for p in v)
    n_pareto_optimal_mappings = sum(len(p.mappings.data) for v in sims.values() for p in v)
    print(f'Took {pmapping_time:.2f} seconds to generate {total_pmappings} partial mappings ({total_pmappings / pmapping_time:.2f} per second). {n_pareto_optimal_mappings} pareto optimal mappings ({n_pareto_optimal_mappings / total_pmappings*100:.2f}% of total).')
    t0 = time.time()
    mappings = join_sims(sims, spec, flattened_architecture)
    join_time = time.time() - t0
    mappings.decompress(decompress_data)
    print(f"Pmappings: {pmapping_time:.2f}. Joining: {join_time:.2f}. Total Pmappings: {total_pmappings}. Total mappings: {mappings.n_pmappings}. Time per pmapping: {pmapping_time / total_pmappings:.2e}")
    pickle.dump(mappings, open(f"cache2/{fname}.pkl", "wb"))
    if return_mappings:
        return mappings
    return objective(mappings.data).min(), mappings

# print(f'Overall area budget: {area_budget * 1e6} mm^2')

parameterization2edp = {}
parameterization2mappings = {}

parameterizations = [
    "Unfused",
    "Fixed-Dataflow",
    "FlashAttention A",
    "FlashAttention B",
    "FFM"
]#, "Unfused"] # "FFM", "Unfused", "FlashAttention"]#, "FlashAttention", "Unfused"]

# TARGET_TECH_NODE = "4nm"
# adder = AladdinAdder(technology=TARGET_TECH_NODE, width=16)
# multiplier = AladdinMultiplier(technology=TARGET_TECH_NODE, width=8)
# mac_area = adder.get_area() + multiplier.get_area()

# # glb_size = 512 * 1024 * 1024 * 8
# # glb = CactiSRAM(technology=TARGET_TECH_NODE, width=1024, depth=glb_size // 1024)
# llb_size = 1 * 1024 * 1024 * 8
# llb = CactiSRAM(technology=TARGET_TECH_NODE, width=128, depth=llb_size // 128)

def binary_search_glb_size(area_remaining):
    print(f"Area remaining: {area_remaining}")
    # Binary search to nearest 1MB
    low = 1
    high = 1024
    while low < high:
        mid = round((low + high) / 2)
        size = mid * 1024 * 1024 * 8
        glb = CactiSRAM(technology=TARGET_TECH_NODE, width=1024, depth=size // 1024)
        if glb.get_area() > area_remaining:
            if mid == high:
                break
            print(f"High {high} -> {mid}")
            high = mid
        else:
            if mid == low:
                break
            print(f"Low {low} -> {mid}")
            low = mid
    return low * 1024 * 1024 * 8

mac_x, mac_y = 128, 128
# for mac_x, mac_y in [(256, 256), (1024, 1024), (1024, 512), (512, 512), (512, 256)]:
# for n_pes in [256, 1, 2, 4, 8, 16, 32, 64, 128, 256]:#2, 4, 8, 16, 32]:
for n_pes in [256, 64]:#, 32, 16, 8, 4, 2, 1]:
    # total_mac_area = mac_area * mac_x * mac_y
    # area_remaining = area_budget - n_pes * (llb.get_area() + total_mac_area)
    # if area_remaining < 0:
    #     continue
    # glb_size = binary_search_glb_size(area_remaining)
    # glb = CactiSRAM(technology=TARGET_TECH_NODE, width=1024, depth=glb_size // 1024)
    # glb_size = 128 * 1024 * 1024 * 8
    # glb = CactiSRAM(technology=TARGET_TECH_NODE, width=1024, depth=glb_size // 1024)
    max_latency = None
    
    # glb_MB = glb_size // 1024 // 1024 // 8
    # llb_MB = llb_size // 1024 // 1024 // 8
    
    print(f"\n\n")
    print(f"=" * 100)
    print(f"MAC dims: {mac_x}x{mac_y}")
    print(f"=" * 100)

    for parameterization in parameterizations: # "fuse"
        while True:
            try:
                x, mappings = get_fused_mappings(
                    spec,
                    n_pes,
                    mac_x,
                    mac_y,
                    parameterization=parameterization,
                    max_latency=max_latency if parameterization != "FlashAttention B" else None
                )
                break
            except TypeError as e:
                max_latency *= 2
                print(f"Error: {e}")
        if "Fixed-Dataflow" not in parameterization:
            max_latency = x
        if x != 0:
            parameterization2edp[f"{parameterization} {n_pes} {mac_x}x{mac_y}"] = x
            parameterization2mappings[f"{parameterization} {n_pes} {mac_x}x{mac_y}"] = mappings

ImportError: cannot import name 'CactiSRAM' from 'hwcomponents_cacti' (/usr/local/lib/python3.13/site-packages/hwcomponents_cacti/__init__.py)

In [2]:
assert False
import matplotlib.pyplot as plt
plt.style.use('default')
# plt.rcParams.update({'font.size': 28})

def plot_default_formatting(ax, grid_axis='both'):
    ax.tick_params(axis='both', which='major')#, labelsize=20)
    ax.tick_params(axis='both', which='minor')#, labelsize=20)
    legend = ax.legend()
    legend.get_frame().set_facecolor('white')
    legend.get_frame().set_edgecolor('black')
    for spine in ax.spines.values():
        spine.set_edgecolor('black')
    if ax.get_legend() is None:
        legend = ax.legend(fontsize=24, ncol=2)
    ax.minorticks_on()
    ax.grid(axis=grid_axis, which='major', linestyle='-', linewidth='0.3', color='gray')
    ax.grid(axis=grid_axis, which='minor', linestyle='--', linewidth='0.1', color='lightgray')
    

def make_bar_chart(
    data,
    title,
    xlabel,
    ylabel,
    y_scale,
    output_file=None,
    normalize: bool = False,
    ylim=(None, None),
    xlim=(None, None),
):
    """
    Create a bar chart from the given data and save it as a PDF.
    """
    plt.figure(figsize=(16, 8))
    
    if isinstance(data, dict) and isinstance(next(iter(data.values())), dict):
        bar_width = 0.8 / len(data)
        keys = list(next(iter(data.values())).keys())
        x = range(len(keys))
        first = next(iter(data.values()))
            
        for i, (label, values) in enumerate(data.items()):
            bar_positions = [pos + i * bar_width for pos in x]
            to_plot = values
            if normalize:
                to_plot = {k: v / first[k] for k, v in values.items()}
            bars = plt.bar(bar_positions, to_plot.values(), width=bar_width, label=label)
        plt.xticks([pos + (len(data) - 1) * bar_width / 2 for pos in x], keys)
        plt.legend(loc='upper right', fontsize=10)
    else:
        keys = list(data.keys())
        bars = plt.bar(keys, data.values())

    # Set logarithmic scale for Y-axis if specified
    if y_scale == 'log':
        plt.yscale('log')

    # Add labels and title
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.ylim(ylim)
    plt.xlim(xlim)

    # Rotate X-axis labels vertically
    plt.xticks(rotation=90)
    
    plot_default_formatting(plt.gca(), grid_axis='y')
    
    if output_file is not None:
        with open(output_file, 'wb') as f:
            plt.savefig(f, format='pdf', bbox_inches='tight')

    # Show the plot
    plt.show()
print()
make_bar_chart(
    parameterization2edp,
    title=None,
    xlabel=None,
    ylabel="EDP",
    y_scale='linear'
)

AssertionError: 

In [ ]:
# # glb_MB = 128
# # sram_MB = 4
# # parameterization = ""

# # cur_area_budget = area_budget
# # glb_size = glb_MB * 1024 * 1024 * 8
# # glb = CactiSRAM(technology="7nm", width=1024, depth=glb_size // 1024)
# # cur_area_budget -= glb.get_area()
# # # for sram_MB in [0.25, 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4]:
# # sram_size = sram_MB * 1024 * 1024 * 8
# # llb = CactiSRAM(technology="7nm", width=128, depth=sram_size // 128)
# # remaining_area = cur_area_budget / 4 - llb.get_area() # Per-MXU
# # mac_dims = int((remaining_area / mac_area) ** 0.5)
# # print(f"Global buffer: {glb_MB} MB, Local buffer: {sram_MB} MB, MAC dims: {mac_dims}x{mac_dims}")
# # print(f'GLB read energy: {glb.read()}. LLB read energy: {llb.read()}')

from fastfusion.mapper.FFM.exploration.mapper_multi_einsum import get_per_tensor_size, get_num_computes
for tensor, size in sorted(get_per_tensor_size(spec).items(), key=lambda x: x[1], reverse=True):
    print(f"{tensor}: {size}")
print(f"Number of computes: {get_num_computes(spec)}")

In [ ]:
from fastfusion.mapper.FFM.exploration.mapper_multi_einsum import get_rank_variable_bounds_for_all_einsums

parameterization2latencycols: list[dict[str, float]] = []
for p, mappings in parameterization2mappings.items():
    mappings._data = mappings.data.sort_values(by="metric_Latency", ascending=True)
    rank_variable_bounds = get_rank_variable_bounds_for_all_einsums(spec)

    row = {
        "Parameterization": p,
    }
    for col in mappings.data.columns:
        print(f'{col}: {mappings.data.iloc[0][col]}')
        # if "Latency" in col:
        # if "metric_Latency" in col:
        if "Latency" in col:
        # if "metric_Energy" in col:
            row[col] = mappings.data.iloc[0][col]
    parameterization2latencycols.append(row)

    # from fastfusion.mapper.FFM.visualization import make_mapping
    # from IPython.display import SVG
    # newmapping = make_mapping(mappings.data.iloc[0], spec.workload.einsum_names, get_rank_variable_bounds_for_all_einsums(spec))
    # for col in mappings.data.columns:
    #     print(f'{col}: {mappings.data.iloc[0][col]}')

    # display(SVG(newmapping.render()))
    
from pandas import DataFrame
df = DataFrame(parameterization2latencycols)
import pandas as pd
pd.set_option('display.max_columns', None)
df
    
# {'n1'}-1 || [GlobalBuffer] T1 sz 0 above 1
# TODO: Re-add -1 to the mapper one eisnum freenig
# compatibility2sims['Matmul1']["{'n1'}-1 || [GlobalBuffer] T1 sz 0 above 1"]
# Above 1: 8192
# Above 2: 8321
# compatibility2sims['Matmul2']["{'n1'}-1 || [GlobalBuffer] T1 sz 0 above 1, [GlobalBuffer] T2 sz 0 above 0"]

In [ ]:
from fastfusion.mapper.FFM.exploration.mapper_multi_einsum import get_rank_variable_bounds_for_all_einsums

mac_dims = int((((area_budget - glb.get_area()) / 4 - llb.get_area()) / mac_area)** 0.5)
mappings = list(parameterization2mappings.values())[0]
mappings._data = mappings.data.sort_values(by="metric_Latency", ascending=True).head()
rank_variable_bounds = get_rank_variable_bounds_for_all_einsums(spec)
from fastfusion.mapper.FFM.visualization import make_mapping
from IPython.display import SVG
newmapping = make_mapping(mappings.data.iloc[0], spec.workload.einsum_names, get_rank_variable_bounds_for_all_einsums(spec))
a = {}
for col in mappings.data.columns:
    print(f'{col}: {mappings.data.iloc[0][col]}')
    if "Latency" in col:
        a[col] = mappings.data.iloc[0][col]
display(SVG(newmapping.render()))

In [ ]:
assert False

from fastfusion.mapper.FFM.exploration.mapper_multi_einsum import get_rank_variable_bounds_for_all_einsums

sram_size = 0.5 * 1024 * 1024 * 8
llb = CactiSRAM(technology="7nm", width=128, depth=sram_size // 128)
mac_dims = int((((area_budget - glb.get_area()) / 4 - llb.get_area()) / mac_area)** 0.5)
mappings = get_fused_mappings(
    spec,
    mac_dims,
    llb,
    glb,
    return_mappings=True,
    parameterization="FFM"
)
mappings._data = mappings.data.sort_values(by="metric_Latency", ascending=True).head()
rank_variable_bounds = get_rank_variable_bounds_for_all_einsums(spec)
from fastfusion.mapper.FFM.visualization import make_mapping
from IPython.display import SVG
newmapping = make_mapping(mappings.data.iloc[0], spec.workload.einsum_names, get_rank_variable_bounds_for_all_einsums(spec))
b = {}
for col in mappings.data.columns:
    print(f'{col}: {mappings.data.iloc[0][col]}')
    if "Latency" in col:
        b[col] = mappings.data.iloc[0][col]
display(SVG(newmapping.render()))

In [ ]:
df = pd.DataFrame([a, b])
df